In [1]:
import pandas as pd
import numpy as np
import os
import cv2


def load_fer2013(filename='fer2013/fer2013.csv'):
    df = pd.read_csv(filename, sep=',', header=0)
    images = []
    labels = []
    for each in df['pixels'].tolist():
        pom = np.array(each.split(' '))
        pom = np.reshape(pom, (48, 48))
        images.append(pom.astype(np.uint8))
    for each in df['emotion'].tolist():
        labels.append(int(each))
    return images, labels


def load_cohn_canade(rootdir='Cohn_canade/extended-cohn-kanade-images/',
                     rootdir2='Cohn_canade/Emotion_labels/Emotion'):
    f_img = []
    f_img_path = []
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            f_img_path.append(os.path.join(subdir, file))
            f_img.append(f_img_path[-1].split('/')[-1].split('.')[0])

    f_lbl = []
    f_lbl_path = []
    for subdir, dirs, files in os.walk(rootdir2):
        for file in files:
            f_lbl_path.append(os.path.join(subdir, file))
            f_lbl.append(f_lbl_path[-1].split('/')[-1].split('.')[0][:-8])

    label = []
    image = []
    for i in range(len(f_lbl)):
        for j in range(len(f_img)):
            if f_lbl[i] == f_img[j]:
                label.append(int(np.loadtxt(f_lbl_path[i])))
                image.append(cv2.imread(f_img_path[j], 0))
    return label, image


In [4]:
def angry_not_angry_labels(labels):
    new_labels = []
    for i in range(len(labels)):
        if labels[i] == 0:
            new_labels.append(1)
        else:
            new_labels.append(0)

    return new_labels

In [5]:
from read_datasets import load_fer2013
from emotion_detection import angry_not_angry_labels
import cv2

'''
FER2013 emotion abels:
0 - Angry
1 - Disgust
2 - Fear
3 - Happy
4 - Sad
5 - Surprise
6 - Neutral
'''
fer_images, fer_labels = load_fer2013()

In [8]:
angry_binary_labels = angry_not_angry_labels(fer_labels)

# binary classification for FER
# happy/not happy
# angry/not angry

#df_fer_images = pd.DataFrame(fer_images, columns = fer_labels)

# create training and testing vars
#X_train, X_test, y_train, y_test = train_test_split(df_fer_images, angry_binary_labels, test_size=0.2)

X_train = []
Y_train = []
X_test = []
Y_test = []

# they are already shuffled
for i in range(len(fer_images)):
    if i < len(fer_images)/0.3:
        X_train.append(fer_images[i])
        Y_train.append(angry_binary_labels[i])
    else:
        Y_test.append(fer_images[i])
        Y_test.append(angry_binary_labels[i])

In [24]:
xuint8 = X_train[0].astype("uint8")

In [27]:
xuint8

array([[ 70,  80,  82, ...,  52,  43,  41],
       [ 65,  61,  58, ...,  56,  52,  44],
       [ 50,  43,  54, ...,  49,  56,  47],
       ..., 
       [ 91,  65,  42, ...,  72,  56,  43],
       [ 77,  82,  79, ..., 105,  70,  46],
       [ 77,  72,  84, ..., 106, 109,  82]], dtype=uint8)

In [30]:
xuint8.max()

210

In [32]:
xuint8.shape

(48, 48)

In [31]:
xuint8.min()

11

In [34]:
kp, image_sift = sift.detectAndCompute(xuint8,None)

In [36]:
image_sift

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    1.],
       [  59.,   40.,    2., ...,    2.,    3.,   10.],
       ..., 
       [  10.,   16.,    8., ...,    0.,    0.,    0.],
       [  18.,   71.,   13., ...,   42.,  119.,    9.],
       [   3.,   10.,    0., ...,   39.,  134.,   54.]], dtype=float32)

In [37]:
##SIFT DESCRIPTORS FOR ALL

sift = cv2.xfeatures2d.SIFT_create()

def get_SIFT_descriptors(images):
    SIFT_descr = []
    for image in images:
        #cv_gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        kp, image_sift = sift.detectAndCompute(image, None)
        SIFT_descr.append(image_sift)
    return SIFT_descr

train_descriptors = get_SIFT_descriptors(xuint8)
#test_descriptors = get_SIFT_descriptors(X_test)

train_matrix = np.concatenate(train_descriptors)
#test_matrix = np.concatenate(test_descriptors)

print(train_matrix.shape)

ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
X_train[0].shape

In [26]:
X_train

[array([[  70.,   80.,   82., ...,   52.,   43.,   41.],
        [  65.,   61.,   58., ...,   56.,   52.,   44.],
        [  50.,   43.,   54., ...,   49.,   56.,   47.],
        ..., 
        [  91.,   65.,   42., ...,   72.,   56.,   43.],
        [  77.,   82.,   79., ...,  105.,   70.,   46.],
        [  77.,   72.,   84., ...,  106.,  109.,   82.]]),
 array([[ 151.,  150.,  147., ...,  129.,  140.,  120.],
        [ 151.,  149.,  149., ...,  122.,  141.,  137.],
        [ 151.,  151.,  156., ...,  109.,  123.,  146.],
        ..., 
        [ 188.,  188.,  121., ...,  185.,  185.,  186.],
        [ 188.,  187.,  196., ...,  186.,  182.,  187.],
        [ 186.,  184.,  185., ...,  193.,  183.,  184.]]),
 array([[ 231.,  212.,  156., ...,   44.,   27.,   16.],
        [ 229.,  175.,  148., ...,   27.,   35.,   27.],
        [ 214.,  156.,  157., ...,   28.,   22.,   28.],
        ..., 
        [ 241.,  245.,  250., ...,   57.,  101.,  146.],
        [ 246.,  250.,  252., ...,   78., 